# Category classifier
The goal of this project is to output a classifier which will predict the review category.

**Customer/Use Case:** Potential customer could be Yelp to aid in tagging of businesses automatically using the input review text
    * Special care will have to be given to which prediction thresholds to use in order to strike a balance between precision & recall for the given use case.

**Approach:** 
1) Data curation and EDA (accomplished in sperate notebooks)
2) Data cleaning
    * Reducing feature and data scope (**Initially PA only**)
4) Featurize the review data
    * First pass will be using **tf-idf**, but additional embeddings could be used as time permits
5)  Budilding out classifiers
    * **Initial POC** will be binary -- restaurant, not-restaurant -- classifier.
    * Secondary will be **multi-class**
    * Plan is to test the following ML classifiers:
      * Logistic Regression
      * Random Forest
      * Gradient Boosted
      * Other classifiers as time permits
6) Evaluate classifers
   * See note above about threshold selection
7) Deployment
   * This is a stretch goal. Would be cool to host on AWS for online input

In [1]:
# Importing all packages including NLTK downloads as necessary
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('ggplot')
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk.corpus
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay, roc_curve, PrecisionRecallDisplay, precision_recall_curve, roc_auc_score
first_run = False
if first_run:
    nltk.download('stopwords')
    nltk.download('punkt')
    nltk.download('wordnet')
import dataprep

## Importing and cleaning data

In [ ]:
data_import = True
if data_import:
    business = pd.read_csv("yelp_dataset/yelp_academic_dataset_business.csv", low_memory=False)
    reviews = pd.read_csv("yelp_dataset/yelp_academic_dataset_review.csv")

In [ ]:
# The review cleaning function take a long time. Filtering dataset before we go further.
clean_business = dataprep.clean_business_data(business)
PA_business = clean_business[clean_business['state'] == 'PA']

filtered_reviews = reviews[reviews['business_id'].isin(PA_business['business_id'])].copy()
PA_reviews = dataprep.clean_review_data(filtered_reviews)

In [ ]:
print(business.shape[0])
print(PA_business.shape[0])
print(reviews.shape[0])
print(PA_reviews.shape[0])

### Filtering for PA only

In [ ]:
# Slicing out only the columns that we may need going forward
PA_business = PA_business[['business_id', 'name', 'category_split']]
PA_reviews = PA_reviews[['review_id', 'business_id', 'text']]

## Joining datasets

In [ ]:
PA_data = PA_business.merge(PA_reviews, how='inner', on='business_id', validate='one_to_many')

### Creating target variable for binary classifier
i.e. Restaurant vs. Not-Restaurant

In [ ]:
PA_data['is_restaurant'] = PA_data.apply(lambda row: row['category_split'].count('restaurants') > 0, 
                                         axis=1)

In [ ]:
# The classes here are imbalanced, but there are so many records, that I'm not concerned right now
PA_data['is_restaurant'].value_counts()

## Getting embeddings from tf-idf for featurization

In [ ]:
# Identifying stopwords from multiple sources
my_stopwords = ['review']
nltk_stop_words = list(nltk.corpus.stopwords.words('english'))
nltk_stop_words = [word.translate(str.maketrans('', '', string.punctuation)) for word in nltk_stop_words]
stopwords = list(set(list(ENGLISH_STOP_WORDS) + my_stopwords + nltk_stop_words))

In [ ]:
# Lemmatizing words
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
        
tf = TfidfVectorizer(strip_accents='unicode',
                     tokenizer=LemmaTokenizer(),
                     stop_words=stopwords,
                     max_features=500) # Setting at 500 for POC. Could be tuned further

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(PA_data['text'].values,
                                                     PA_data['is_restaurant'].values, 
                                                     test_size=0.25, 
                                                     random_state=43)

In [ ]:
# Could use n-grams here
tfidf = tf.fit_transform(X_train) #ngram_range=(1, 2)) #Including uni and bi-grams
tfidf_test = tf.transform(X_test)

## Building classifiers
Note that I'm not cross-validating my scores for the sake of time

### Logistic Regression

In [ ]:
#LogisticRegressionCV regularlizes by default so we set C really high to avoid this
lr = LogisticRegression(random_state=43, C=1000000, max_iter=400)
lr.fit(tfidf, y_train)
y_pred = lr.predict(tfidf_test)
y_pred_score = lr.predict_proba(tfidf_test)

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_pred_score[:,1], pos_label=lr.classes_[1])
prec, recall, _ = precision_recall_curve(y_test, y_pred_score[:,1], pos_label=lr.classes_[1])
auc = roc_auc_score(y_test, y_pred_score[:,1])
fig, axs = plt.subplots(1, 2, figsize=(8, 3))
RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=auc).plot(axs[0])
PrecisionRecallDisplay(precision=prec, recall=recall).plot(axs[1])
# cm = confusion_matrix(y_test, y_pred)
# ConfusionMatrixDisplay(cm).plot()
axs[0].plot((0,1), (0,1), ls='--', c='gray')
fig.suptitle('Logistic Regression')
plt.tight_layout()
plt.show()

### Random forest classifier

This takes a really long time to train, so I'm going to subsample the data and do some averaging for the metrics.

In [ ]:
# I'm selecting a few subsample sets from the training set, given the size of the data and training time.
# Ideally I would do some K-Fold cross-validation here and later hyper-parameter tuning, but for time-constraints
# I decided against it.
# sklearn.model_selection.ShuffleSplit could be a good way to do this as well
num_slices = 5
max_dataset = 100000
indices_dict = dict()
for i in range(num_slices):
    indices_dict[i] = np.random.choice(tfidf.shape[0], size=max_dataset, replace=True)

In [ ]:
# Starting with default values
fprs = []
tprs = []
precs = []
recalls = []
aucs = []

for i in range(num_slices):
    rf = RandomForestClassifier(bootstrap=True, n_jobs=-1, random_state=42)
    rf.fit(tfidf[indices_dict[0]], y_train[indices_dict[0]])
    y_pred_score = rf.predict_proba(tfidf_test)
    fpr, tpr, _ = roc_curve(y_test, y_pred_score[:,1], pos_label=rf.classes_[1])
    prec, recall, _ = precision_recall_curve(y_test, y_pred_score[:,1], pos_label=rf.classes_[1])
    auc = roc_auc_score(y_test, y_pred_score[:,1])
    fprs.append(fpr)
    tprs.append(tpr)
    precs.append(prec)
    recalls.append(recall)
    aucs.append(auc)

In [ ]:
auc = roc_auc_score(y_test, y_pred_score[:,1])
fig, axs = plt.subplots(1, 2, figsize=(8, 3))
for i in range(num_slices):
    RocCurveDisplay(fpr=fprs[i], tpr=tprs[i], roc_auc=aucs[i]).plot(axs[0])
    PrecisionRecallDisplay(precision=precs[i], recall=recalls[i]).plot(axs[1])
RocCurveDisplay(fpr=pd.DataFrame(fprs).mean(axis=0).values, 
            tpr=pd.DataFrame(tprs).mean(axis=0).values, 
            roc_auc=np.array(aucs).mean()).plot(axs[0], 
                                                label='Mean AUC = {}'.format(round(np.array(aucs).mean(),
                                                                                  2)))
axs[0].plot((0,1), (0,1), ls='--', c='gray')
fig.suptitle('Random Forest')
plt.tight_layout()
plt.show()

# Sample Texts

In [ ]:
test_texts = pd.DataFrame(['best burritos in town close to my house service is excellent',
              'i hate this place its the worst service ive ever experienced i will never go back',
              'i love buying my car parts here',
              'ive been going to this dentist for 5 years and every time ive had a good experience'], columns = ['text'])

sample_tfidf = tf.transform(test_texts['text'])

test_texts['lr_predict'] = lr.predict_proba(sample_tfidf)[:,1]
test_texts['rf_predict'] = rf.predict_proba(sample_tfidf)[:,1]
test_texts

# Conclusions
Based on the above, it appears that the LogisticRegression classifier performes better than the RandomForest classifer with an AUC of 0.97 compared to 0.96. However, these are so close, that it could be noise. Both classifiers perform very well over random guessing.

There are many ways that this work could be taken further:
1) Developing a multi-class classifier (not just restaurants)
2) Performing K-fold tests on both models to be sure of the AUC
3) Tuning the hyperparameters
4) Training on a larger dataset (although this is unlikely to impact the model much)
5) Adding other features to the models (zipcode, number of patrons, words in name, review_stars, various other attributes, etc.)
6) Featurzing with n-grams instead of only unigrams
7) Using different word embeddings (Word2Vec)

## Next Steps
The next step in the process could be to select an appropriate threshold for prediction before feeding a user a prompt, taking into account the cost of a FP and FN. For example, if we determine that prompting a user to tag a review as "restaurant" when it is in fact a gas station, may lead to user churn and or dirty data. If we wish to avoid FPs we may choose a higher Precision over Recall for the final deployment.

In [ ]:
#f1 score

In [ ]:
# filtered_reviews = reviews[reviews['business_id'].isin(filtered_data['business_id'])].copy()
# # drop duplicates
# filtered_reviews.drop_duplicates('review_id', inplace=True)
# # Removing any additional duplicate reviews from same user with same text and business
# filtered_reviews.drop_duplicates(['business_id', 'user_id', 'text'], inplace=True)
# # filtered_reviews.reset_index(drop=True, inplace=True)
# # Cleaning up text field
# filtered_reviews['text'] = filtered_reviews['text'].str.lstrip('u').str.strip().str.lower().str.translate(str.maketrans('-', ' ', string.punctuation))
# # Removing escaped characters that show up literally
# filtered_reviews['text'] = filtered_reviews['text'].str.translate(str.maketrans(dict([(chr(char), ' ') for char in range(1, 32)])))
# # Removing >1 whitespace chars
# def clean_whitespace(text):
#     return ' '.join(text.split())
# filtered_reviews['text'] = filtered_reviews.apply(lambda row: clean_whitespace(row['text']), axis=1)
# # Dropping few reviews that are NaN
# filtered_reviews = filtered_reviews[~filtered_reviews['text'].isna()]

In [ ]:
# data_import = True
# if data_import:
#     business = pd.read_csv("yelp_dataset/yelp_academic_dataset_business.csv", low_memory=False)
# business.drop_duplicates('business_id', inplace=True)
# biz_columns_to_drop = ['hours', 'attributes', 'attributes.BusinessParking', 'attributes.HairSpecializesIn', 'attributes.RestaurantsAttire', 'attributes.BestNights',
#                        'attributes.Open24Hours', 'attributes.Music', 'attributes.AgesAllowed', 'attributes.BusinessAcceptsCreditCards', 'attributes.DietaryRestrictions']
# for column in biz_columns_to_drop:
#     business.drop(column, inplace=True, axis=1)
# business.reset_index(inplace=True, drop=True)
# # Likely not necessary, but updating unicode string import issue leading to leading 'u's
# unicode_fix_columns = ['attributes.Alcohol', 'attributes.Smoking', 'attributes.NoiseLevel', 'attributes.BYOBCorkage']

# for column in unicode_fix_columns:
#     business[column] = business[column].str.lstrip('u').str.strip()

# # Update names to strip punctuation and lowercase
# business['name'] = business['name'].str.strip().str.lower().str.translate(str.maketrans('', '', string.punctuation))
# # Updating corkage to fix duplicate values
# business['attributes.BYOBCorkage'] = business['attributes.BYOBCorkage'].str.split('_').str[0].value_counts()

# def split_categories(categories):
#     try:
#         output = categories.split(',')
#         return [x.strip().lower() for x in output]
#     except:
#         return ""
        
# business['category_split'] = business.apply(lambda row: split_categories(row['categories']), axis=1)

# categories_to_check = ['restaurants']
# business['applicable_categories'] = business['category_split'].apply(lambda x : 1 if any(i in x for i in categories_to_check) else 0)